In [1]:
import os
import torch
from torch.cuda.amp import autocast, GradScaler
from torch_geometric.data import Data
import pandas as pd
import numpy as np
import torch.nn as nn
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score
import copy
import random
import multiprocessing as mp
from torch_geometric.data import HeteroData
import torch
from torch_geometric.nn import GCNConv
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, precision_recall_curve, roc_curve, auc
import matplotlib.pyplot as plt
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv
from torch_geometric.loader import DataLoader  # For loading graphs in batches
from torch_geometric.utils import negative_sampling  # For handling link prediction tasks
from torch_geometric.utils import to_undirected
import matplotlib.pyplot as plt
from torch.optim.lr_scheduler import ReduceLROnPlateau
from datetime import datetime
import torch.optim.lr_scheduler as lr_scheduler


### Preliminaries:

In [2]:
# Load node data from csv
s_df = pd.read_csv('/data/servilla/DT_HGNN/Model/Nodes/s_emb_full_183.csv', index_col=0)
p_df = pd.read_csv('/data/servilla/DT_HGNN/Model/Nodes/p_emb_full_237197.csv', index_col=0)


In [3]:
# Load edge data from csv
tp_s_df = pd.read_csv('/data/servilla/DT_HGNN/Model/Edges/distributed_combined_tp_s_edges_13340.csv')
ppi_df = pd.read_csv('/data/servilla/DT_HGNN/Model/Edges/combined_ppi_edges_full.csv')
ssi_df = pd.read_csv('/data/servilla/DT_HGNN/Model/Edges/combined_ssi_edges_full.csv')

In [4]:
# Load device of available GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if device.type == 'cuda':
    device = torch.device("cuda:0")
    print(device)
    

cuda:0


### Function Definitions:

In [5]:
# Function to get the current date and time for the file name and log
def get_timestamp():
    return datetime.now().strftime('%Y-%m-%d_%H-%M-%S')

In [6]:
# Inspect and clean the data, converts non-numeric columns to numeric and fills NaN values with 0
def inspect_and_clean(df):
    non_numeric_columns = df.select_dtypes(exclude=[np.number]).columns
    print(f"Non-numeric columns: {non_numeric_columns}")
    if len(non_numeric_columns) > 0:
        df[non_numeric_columns] = df[non_numeric_columns].apply(pd.to_numeric, errors='coerce')
    df = df.fillna(0)
    return df


In [7]:
# Apply transformations in batches, this can be useful when dealing with large 
# datasets that may not fit into memory or GPU all at once. 
def transform_in_batches(features, transform_layer, batch_size=10000):
    num_samples = features.shape[0]
    print(f"Number of samples: {num_samples}")
    transformed_features = []
    for i in range(0, num_samples, batch_size):
        batch = features[i:i + batch_size]
        batch_tensor = torch.tensor(batch, dtype=torch.float).to(device)
        transformed_batch = transform_layer(batch_tensor)
        transformed_features.append(transformed_batch.detach().cpu().numpy())  # Use detach() before numpy()
    return np.vstack(transformed_features) # Stack arrays in sequence vertically (row wise)


In [8]:
def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if you are using multi-GPU
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


In [9]:
def split_data(df, train_size=0.8, val_size=0.1, test_size=0.1, random_state=42):
    # Shuffle and split the data into train, validation, and test sets at once
    # Calculate the remaining size after the train split (val + test)
    train_df, temp_df = train_test_split(df, train_size=train_size, random_state=random_state)
    
    # Calculate the size for validation and test splits from the remaining data
    val_test_ratio = val_size / (val_size + test_size)
    
    # Split the remaining temp_df into validation and test sets
    val_df, test_df = train_test_split(temp_df, train_size=val_test_ratio, random_state=random_state)
    
    return train_df, val_df, test_df


In [10]:
def apply_correct_mapping(df, source_mapping, target_mapping):
    # Apply the mappings to 'source' and 'target'
    df['mapped_source'] = df['source'].map(source_mapping)
    df['mapped_target'] = df['target'].map(target_mapping)

    # Identify rows where mapping failed (NaN values)
    unmapped_sources = df[df['mapped_source'].isna()]['source'].unique()
    unmapped_targets = df[df['mapped_target'].isna()]['target'].unique()

    # Log or print unmapped elements
    if len(unmapped_sources) > 0:
        print(f"Unmapped sources: {unmapped_sources}")
    if len(unmapped_targets) > 0:
        print(f"Unmapped targets: {unmapped_targets}")

    # Remove rows where mapping failed (NaN values)
    df.dropna(subset=['mapped_source', 'mapped_target'], inplace=True)

    # Replace original 'source' and 'target' with mapped values and drop the extra columns
    df['source'] = df['mapped_source']
    df['target'] = df['mapped_target']
    df.drop(columns=['mapped_source', 'mapped_target'], inplace=True)
    
    return df


In [11]:
# Convert edges to undirected and combine symmetric labels using 'reduce'
def undirected(edges_df, label_column, reduce='mean'):
    # Ensure 'source' and 'target' are numeric
    edges_df['source'] = pd.to_numeric(edges_df['source'], errors='coerce')
    edges_df['target'] = pd.to_numeric(edges_df['target'], errors='coerce')

    # Check for NaN values (if any elements could not be converted)
    if edges_df[['source', 'target']].isna().any().any():
        raise ValueError("Some source or target nodes could not be converted to numeric values.")

    # Convert to torch tensor
    edge_index = torch.tensor(edges_df[['source', 'target']].values.T, dtype=torch.long)

    # Extract the labels (weights) as a tensor
    edge_attr = torch.tensor(edges_df[label_column].values, dtype=torch.float)

    # Convert to undirected edges and combine labels
    edge_index, edge_attr = to_undirected(edge_index, edge_attr=edge_attr, reduce=reduce)

    # Create a new DataFrame with expanded edges and combined labels
    new_edges_df = pd.DataFrame({
        'source': edge_index[0].numpy(),
        'target': edge_index[1].numpy(),
        label_column: edge_attr.numpy()  # Store the combined labels
    })
    
    return new_edges_df


In [12]:
# For testing, we can remove percentage of edges from the each of the training sets
def remove_edges(edges, removal_percentage):
    num_edges = edges.size(1)
    num_to_remove = int(removal_percentage * num_edges)
    indices_to_keep = torch.randperm(num_edges)[:-num_to_remove]  # Randomly select edges to keep
    return edges[:, indices_to_keep]

### Code to manipulate data:

In [13]:
set_seed(42)


In [14]:
s_df = inspect_and_clean(s_df)
p_df = inspect_and_clean(p_df)


Non-numeric columns: Index([], dtype='object')
Non-numeric columns: Index([], dtype='object')


In [15]:
# Convert features to numpy arrays
s_features = s_df.values
p_features = p_df.values


In [16]:
# Check the shape of the features
print(f"Shape of s_features: {s_features.shape}")
print(f"Shape of p_features: {p_features.shape}")


Shape of s_features: (183, 1536)
Shape of p_features: (12301, 2048)


In [17]:
# Create separate mappings
protein_mapping = {node_id: i for i, node_id in enumerate(p_df.index)}
substrate_mapping = {node_id: i for i, node_id in enumerate(s_df.index)}


In [18]:
# Apply mappings to the full edge DataFrames before splitting
tp_s_df = apply_correct_mapping(tp_s_df, protein_mapping, substrate_mapping)
ppi_df = apply_correct_mapping(ppi_df, protein_mapping, protein_mapping)
ssi_df = apply_correct_mapping(ssi_df, substrate_mapping, substrate_mapping)


Unmapped sources: ['Q9ZFK1' 'P70910' 'P70911' ... 'A4G0Y3' 'A4FZL6' 'A4FZY5']
Unmapped targets: ['Q8VS69' 'P70911' 'P70910' ... 'P36370' 'A4FWY9' 'A4FW54']


In [19]:
tp_s_df = undirected(tp_s_df, label_column='label', reduce='mean')
ppi_df = undirected(ppi_df, label_column='label', reduce='mean')
ssi_df = undirected(ssi_df, label_column='label', reduce='mean')


In [20]:
# Split data for each edge type
ppi_train_df, ppi_val_df, ppi_test_df = split_data(ppi_df)
ssi_train_df, ssi_val_df, ssi_test_df = split_data(ssi_df)
tp_s_train_df, tp_s_val_df, tp_s_test_df = split_data(tp_s_df)


In [21]:
# Create edge index tensors
train_edges_tp_s = torch.tensor(tp_s_train_df[['source', 'target']].values.T, dtype=torch.long)
val_edges_tp_s = torch.tensor(tp_s_val_df[['source', 'target']].values.T, dtype=torch.long)
test_edges_tp_s = torch.tensor(tp_s_test_df[['source', 'target']].values.T, dtype=torch.long)

train_edges_ppi = torch.tensor(ppi_train_df[['source', 'target']].values.T, dtype=torch.long)
val_edges_ppi = torch.tensor(ppi_val_df[['source', 'target']].values.T, dtype=torch.long)
test_edges_ppi = torch.tensor(ppi_test_df[['source', 'target']].values.T, dtype=torch.long)

train_edges_ssi = torch.tensor(ssi_train_df[['source', 'target']].values.T, dtype=torch.long)
val_edges_ssi = torch.tensor(ssi_val_df[['source', 'target']].values.T, dtype=torch.long)
test_edges_ssi = torch.tensor(ssi_test_df[['source', 'target']].values.T, dtype=torch.long)


In [22]:
# Convert the labels to tensors
train_labels_tp_s = torch.tensor(tp_s_train_df['label'].values, dtype=torch.float)
val_labels_tp_s = torch.tensor(tp_s_val_df['label'].values, dtype=torch.float)
test_labels_tp_s = torch.tensor(tp_s_test_df['label'].values, dtype=torch.float)

train_labels_ppi = torch.tensor(ppi_train_df['label'].values, dtype=torch.float)
val_labels_ppi = torch.tensor(ppi_val_df['label'].values, dtype=torch.float)
test_labels_ppi = torch.tensor(ppi_test_df['label'].values, dtype=torch.float)

train_labels_ssi = torch.tensor(ssi_train_df['label'].values, dtype=torch.float)
val_labels_ssi = torch.tensor(ssi_val_df['label'].values, dtype=torch.float)
test_labels_ssi = torch.tensor(ssi_test_df['label'].values, dtype=torch.float)


In [23]:
# Split training data into positive samples
positive_tp_s_train_df = tp_s_train_df[tp_s_train_df['label'] == 1]

train_edges_tp_s_positive = torch.tensor(positive_tp_s_train_df[['source', 'target']].values.T, dtype=torch.long)

positive_ppi_train_df = ppi_train_df[ppi_train_df['label'] == 1]

train_edges_ppi_positive = torch.tensor(positive_ppi_train_df[['source', 'target']].values.T, dtype=torch.long)

positive_ssi_train_df = ssi_train_df[ssi_train_df['label'] == 1]

train_edges_ssi_positive = torch.tensor(positive_ssi_train_df[['source', 'target']].values.T, dtype=torch.long)


In [24]:
# For testing, we can remove percentage of edges from the training set
# Define the percentage of edges to remove
ppi_removal_percentage = 1.0  # Remove 20% of PPI edges
ssi_removal_percentage = 1.0  # Remove 20% of SSI edges
tp_s_removal_percentage = 1.0  # Remove 20% of tp_s edges

# Remove a percentage of PPI edges
train_edges_ppi_positive_reduced = remove_edges(train_edges_ppi_positive, ppi_removal_percentage)

# Remove a percentage of SSI edges
train_edges_ssi_positive_reduced = remove_edges(train_edges_ssi_positive, ssi_removal_percentage)

# Remove a percentage of tp_s edges
train_edges_tp_s_positive_reduced = remove_edges(train_edges_tp_s_positive, tp_s_removal_percentage)

train_edges_tp_s_reduced = remove_edges(train_edges_tp_s, tp_s_removal_percentage)

val_edges_tp_s_reduced = remove_edges(val_edges_tp_s, tp_s_removal_percentage)


In [25]:
# Converting the features to tensors (first convert to numpy arrays)
s_np = s_df.values
p_np = p_df.values

s_features_tensor = torch.tensor(s_np, dtype=torch.float).to(device)
p_features_tensor = torch.tensor(p_np, dtype=torch.float).to(device)


In [26]:
from torch_geometric.data import HeteroData

data = HeteroData()

# Assign node features
data['protein'].x = p_features_tensor
data['substrate'].x = s_features_tensor

# Create HeteroData for training
train_data = HeteroData()

# Add node features (assumed the same for training and validation)
train_data['protein'].x = data['protein'].x
train_data['substrate'].x = data['substrate'].x

# Add training edges for tp_s, ppi, and ssi
train_data['protein', 'interacts_with', 'substrate'].edge_index = train_edges_tp_s_reduced
train_data['protein', 'interacts_with', 'protein'].edge_index = train_edges_ppi_positive
train_data['substrate', 'interacts_with', 'substrate'].edge_index = train_edges_ssi_positive


In [27]:
# Create HeteroData for validation
val_data = HeteroData()

# Add node features
val_data['protein'].x = data['protein'].x
val_data['substrate'].x = data['substrate'].x

# Add validation edges
val_data['protein', 'interacts_with', 'substrate'].edge_index = val_edges_tp_s
val_data['protein', 'interacts_with', 'protein'].edge_index = val_edges_ppi
val_data['substrate', 'interacts_with', 'substrate'].edge_index = val_edges_ssi


In [28]:
# Create HeteroData for testing
test_data = HeteroData()

# Add node features (assumed same as training/validation)
test_data['protein'].x = data['protein'].x
test_data['substrate'].x = data['substrate'].x

# Add test edges
test_data['protein', 'interacts_with', 'substrate'].edge_index = test_edges_tp_s
test_data['protein', 'interacts_with', 'protein'].edge_index = test_edges_ppi
test_data['substrate', 'interacts_with', 'substrate'].edge_index = test_edges_ssi


In [29]:
class GraphSAGEModel(torch.nn.Module):
    def __init__(self, protein_input_dim, substrate_input_dim, hidden_dim, output_dim, dropout_prob=0.5):
        super(GraphSAGEModel, self).__init__()
        
        # Transformation layers to match input dimensions
        self.transform_p = Linear(protein_input_dim, hidden_dim)  # Protein: 2048 -> 128
        self.transform_s = Linear(substrate_input_dim, hidden_dim)  # Substrate: 1536 -> 128
        
        # GraphSAGE convolution layers
        self.sage_conv1 = SAGEConv(hidden_dim, hidden_dim)
        self.sage_conv2 = SAGEConv(hidden_dim, output_dim)
        
        # Dropout layer
        self.dropout = torch.nn.Dropout(p=dropout_prob)  # Dropout probability, usually set to 0.5
        
    def forward(self, data):
        # Get the protein and substrate features from HeteroData
        p_features = data['protein'].x
        s_features = data['substrate'].x
        
        # Apply transformations to ensure both node types have the same dimensionality
        p_transformed = self.transform_p(p_features)  # Protein features: 2048 -> 128
        s_transformed = self.transform_s(s_features)  # Substrate features: 1536 -> 128

        # Normalize the transformed features
        p_transformed = (p_transformed - p_transformed.mean(dim=0)) / p_transformed.std(dim=0)
        s_transformed = (s_transformed - s_transformed.mean(dim=0)) / s_transformed.std(dim=0)
        
        # Combine the transformed features
        all_features = torch.cat([p_transformed, s_transformed], dim=0)

        # # Print edge shapes to verify their dimensions
        # print(f"Protein-Substrate edge shape: {data['protein', 'interacts_with', 'substrate'].edge_index.shape}")

        
        # Apply GraphSAGE convolution layers
        out = F.relu(self.sage_conv1(all_features, data['protein', 'interacts_with', 'substrate'].edge_index))
        out = self.dropout(out)  # Apply dropout after the first convolution layer
        
        # Second GraphSAGE convolution
        out = self.sage_conv2(out, data['protein', 'interacts_with', 'substrate'].edge_index)
        
        return out


In [30]:
# Define the device (GPU if available, otherwise CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Initialize the model and move it to the device (GPU or CPU)
model = GraphSAGEModel(protein_input_dim=2048, substrate_input_dim=1536, hidden_dim=128, output_dim=64).to(device)

# Move your HeteroData to the same device
data = data.to(device)

# Move labels and data to device
train_labels_tp_s = train_labels_tp_s.to(device)
val_labels_tp_s = val_labels_tp_s.to(device)
train_data = train_data.to(device)
val_data = val_data.to(device)

# Initialize optimizer and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)
criterion = torch.nn.BCEWithLogitsLoss()

early_stopping_patience = 10
best_val_loss = float('inf')
patience_counter = 0


In [31]:
# Set up early stopping and checkpointing
early_stopping_patience = 10
best_val_loss = float('inf')
patience_counter = 0

# Initialize lists to store training metrics
train_losses = []
val_losses = []
train_aucs = []
val_aucs = []

# Define your hyperparameters in a dictionary
hyperparameters = {
    'learning_rate': 1e-3,
    'weight_decay': 1e-5,
    'hidden_dim': 128,
    'output_dim': 64,
    'dropout_prob': 0.5,
    'optimizer': 'Adam'
}

# Initialize optimizer and learning rate scheduler
optimizer = torch.optim.Adam(model.parameters(), lr=hyperparameters['learning_rate'], weight_decay=hyperparameters['weight_decay'])
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.8, patience=100)

# Training loop
model.train()
for epoch in range(1000):
    optimizer.zero_grad()

    # Print the size of train_edges_tp_s_reduced
    print(f'Epoch {epoch}, train_edges_tp_s_reduced size: {train_edges_tp_s_reduced.size()}')
    
    # Forward pass
    train_out = model(train_data)
    source_embeddings = train_out[train_edges_tp_s[0]]
    target_embeddings = train_out[train_edges_tp_s[1]]
    edge_scores = (source_embeddings * target_embeddings).sum(dim=1)
    train_loss = criterion(edge_scores, train_labels_tp_s)

    # Backward pass and optimization
    train_loss.backward()
    optimizer.step()

    # Save training loss
    train_losses.append(train_loss.item())
    
    # Compute AUC for training data
    train_probs = torch.sigmoid(edge_scores).detach().cpu().numpy()
    train_labels_np = train_labels_tp_s.cpu().numpy()
    train_auc = roc_auc_score(train_labels_np, train_probs)
    train_aucs.append(train_auc)

    print(f'Epoch {epoch}, Training Loss: {train_loss.item()}, Training AUC: {train_auc}, LR: {scheduler.get_last_lr()[0]}')

    # Validation pass
    model.eval()
    with torch.no_grad():
        val_out = model(val_data)
        val_source_embeddings = val_out[val_edges_tp_s[0]]
        val_target_embeddings = val_out[val_edges_tp_s[1]]
        val_edge_scores = (val_source_embeddings * val_target_embeddings).sum(dim=1)
        val_loss = criterion(val_edge_scores, val_labels_tp_s)

        # Save validation loss
        val_losses.append(val_loss.item())

        # Compute AUC for validation data
        val_probs = torch.sigmoid(val_edge_scores).cpu().numpy()
        val_labels_np = val_labels_tp_s.cpu().numpy()
        val_auc = roc_auc_score(val_labels_np, val_probs)
        val_aucs.append(val_auc)

        print(f'Validation Loss: {val_loss.item()}, Validation AUC: {val_auc}')
        
        # Early Stopping Logic
        if val_loss.item() < best_val_loss:
            best_val_loss = val_loss.item()
            patience_counter = 0
        else:
            patience_counter += 1
            print(f'Epoch {epoch}, Validation Loss: {val_loss.item():.4f}. Patience Counter: {patience_counter}')

        if patience_counter >= early_stopping_patience:
            print(f'Early stopping triggered at epoch {epoch}')
            break

    # Step the scheduler with the validation loss
    scheduler.step(val_loss.item())

    model.train()  # Switch back to training mode

# Add timestamp to checkpoint path at the end of training
final_checkpoint_path = f'/data/servilla/DT_HGNN/Model/Model_Checkpoints/GraphSAGE/{get_timestamp()}_best_model.pt'

# Save the final model checkpoint with all the data
checkpoint = {
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'scheduler_state_dict': scheduler.state_dict(),  # Save scheduler state
    'train_losses': train_losses,
    'val_losses': val_losses,
    'train_aucs': train_aucs,
    'val_aucs': val_aucs,
    'best_val_loss': best_val_loss,
    'epoch': epoch,
    'timestamp': get_timestamp(),
    'hyperparameters': hyperparameters
}

torch.save(checkpoint, final_checkpoint_path)
print(f"Final model and training data saved as {final_checkpoint_path}")

# Plot Training and Validation Loss
plt.figure(figsize=(10, 6))
plt.plot(train_losses, label='Training Loss')
plt.plot(val_losses, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss Over Time')
plt.legend()
plt.show()

# Plot Training and Validation AUC
plt.figure(figsize=(10, 6))
plt.plot(train_aucs, label='Training AUC')
plt.plot(val_aucs, label='Validation AUC')
plt.xlabel('Epochs')
plt.ylabel('AUC')
plt.title('Training and Validation AUC Over Time')
plt.legend()
plt.show()

# --------------- TEST SET EVALUATION ---------------

# Load only the model's state_dict for testing
checkpoint = torch.load(final_checkpoint_path)
model.load_state_dict(checkpoint['model_state_dict'])
print('Best model loaded for testing.')

# Move test data to device
test_data = test_data.to(device)
test_labels_tp_s = test_labels_tp_s.to(device)

# Evaluate the model on the test set
model.eval()  # Switch to evaluation mode
with torch.no_grad():  # Disable gradient calculation for evaluation
    test_out = model(test_data)
    
    # Extract source and target node embeddings for the tp_s test edges
    test_source_embeddings = test_out[test_edges_tp_s[0]]
    test_target_embeddings = test_out[test_edges_tp_s[1]]
    
    # Compute the edge scores (dot product or another method)
    test_edge_scores = (test_source_embeddings * test_target_embeddings).sum(dim=1)
    
    # Compute test loss using edge scores and test labels
    test_loss = criterion(test_edge_scores, test_labels_tp_s)
    print(f'Test Loss: {test_loss.item()}')

    # Test accuracy
    test_predictions = torch.sigmoid(test_edge_scores) > 0.5
    accuracy = (test_predictions == test_labels_tp_s).sum().item() / test_labels_tp_s.size(0)
    print(f'Test Accuracy: {accuracy * 100:.2f}%')

    # Test AUC
    test_probs = torch.sigmoid(test_edge_scores).cpu().numpy()
    test_labels_np = test_labels_tp_s.cpu().numpy()
    test_auc = roc_auc_score(test_labels_np, test_probs)
    print(f'Test AUC: {test_auc}')


Epoch 0, train_edges_tp_s_reduced size: torch.Size([2, 0])


ValueError: continuous format is not supported